In [55]:
from scapy.all import Ether, ARP
from p4.v1 import p4runtime_pb2
import google.protobuf.text_format as tf
import glob
import sys

In [56]:
log_files = glob.glob("*.log")
print(log_files)
if len(log_files) == 0:
    print("can not find any *.log file")
    sys.exit(1)
log_file = log_files[0]
print(f"Parsing {log_file}")

packet_msgs = []
with open(log_file) as logf:
    pkt_msg = ""
    indent = 0
    msg = None
    for line in logf:
        if "StreamChannel >> OUTBOUND" in line:
            msg = "out"
            continue
        elif "StreamChannel << INBOUND" in line:
            msg = "in"
            continue
        if msg:
            if "{\n" in line:
                indent += 1
                pkt_msg += line
                continue
            elif "}\n" in line:
                pkt_msg += line
                indent -= 1
                if indent == 0:
                    packet_msgs.append((msg, pkt_msg))
                    msg = None
                    pkt_msg = ""
            elif indent != 0:
                pkt_msg += line


['grpc___10_93_1_252_9339_device_id_1_8907898971417070689.log']
Parsing grpc___10_93_1_252_9339_device_id_1_8907898971417070689.log


In [59]:

result = ""
for pkt_msg in packet_msgs:
    if pkt_msg[0] == "in" and "packet {" in pkt_msg[1]:
        p4_pkt_msg = p4runtime_pb2.StreamMessageResponse()
    elif pkt_msg[0] == "out" and "packet {" in pkt_msg[1]:
        p4_pkt_msg = p4runtime_pb2.StreamMessageRequest()
    tf.Merge(pkt_msg[1], p4_pkt_msg)
    in_port = None
    for md in p4_pkt_msg.packet.metadata:
        if md.metadata_id == 1:
            in_port = int.from_bytes(md.value, 'big')
    pkt = Ether(p4_pkt_msg.packet.payload)
    if ARP not in pkt:
        continue
    result += f"Direction: {pkt_msg[0].upper()}, port: {in_port}\n"
    result += f"{pkt[Ether].src} -> {pkt[Ether].dst} "
    result += pkt[ARP].summary()
    result += "\n"
 
with open("result.txt", "w") as f:
    f.write(result)

print("Results are stored in the result.txt")


Results are stored in the result.txt
